# Model development using MIMIC-IV EMR data only (Strategies 0-3)

1. Summary statistics
2. Feature selection (to add)
3. Model development
4. Hyperparameter tuning (to add)
5. Evaluation of the final model and error analysis (to add)

<img src="../results/class distribution.jpeg" alt="Groups" style="width: 400px;"/>

In [1]:
import numpy as np
import pandas as pd
import utils
from time import time

import copy, math, os, pickle, time 
import scipy.stats as ss

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.pipeline import Pipeline

from sklearn.calibration import CalibratedClassifierCV, calibration_curve

from sklearn.metrics import average_precision_score, roc_auc_score, accuracy_score, f1_score, precision_recall_curve

# To show all columns in a dataframe
pd.options.display.max_info_columns=250
pd.options.display.max_columns=500

# To make pretty plots
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('seaborn-ticks')
sns.set_style('ticks')
plt.rcParams['figure.figsize'] = (6, 4)
plt.rcParams['axes.titlesize'] = 22
plt.rcParams['axes.labelsize'] = 20
plt.rcParams['xtick.labelsize'] = 16
plt.rcParams['ytick.labelsize'] = 16

%matplotlib inline

### Load and prepare the data
* For a simple model predicting PMV add "S0" to filename and set label to "over72h"
* For strategy S1 add "S1" to filename and set label to "over72h"
* For strategy S2 add "S2" to filename and set label to "over72h"
* For strategy S3 add "S3" to filename and set label to "good_outcome"

In [2]:
df_train = pd.read_csv("../data/mimic-emr-ft98-train-S0.csv")
df_train.drop(columns=["starttime", "endtime"], inplace=True)

label = "over72h"

print(df_train.shape)
df_train.head()

(10121, 103)


,stay_id,admission_location,insurance,language,ethnicity,marital_status,gender,age,hours_in_hosp_before_intubation,weight,height,co2_total_max,co2_total_avg,co2_total_min,ph_max,ph_avg,ph_min,lactate_max,lactate_avg,lactate_min,pao2fio2ratio,heart_rate_max,heart_rate_avg,heart_rate_min,mbp_max,mbp_avg,mbp_min,mbp_ni_max,mbp_ni_avg,mbp_ni_min,resp_rate_max,resp_rate_avg,resp_rate_min,temp_max,temp_avg,temp_min,spo2_max,spo2_avg,spo2_min,glucose_max,glucose_avg,glucose_min,vasopressin,epinephrine,dobutamine,norepinephrine,phenylephrine,dopamine,count_of_vaso,fio2_max,fio2_avg,fio2_min,peep_max,peep_avg,peep_min,plateau_pressure_max,plateau_pressure_avg,plateau_pressure_min,rrt,sinus_rhythm,neuroblocker,congestive_heart_failure,cerebrovascular_disease,dementia,chronic_pulmonary_disease,rheumatic_disease,mild_liver_disease,diabetes_without_cc,diabetes_with_cc,paraplegia,renal_disease,malignant_cancer,severe_liver_disease,metastatic_solid_tumor,aids,SOFA,respiration,coagulation,liver,cardiovascular,cns,renal,apsiii,hr_score,mbp_score,temp_score,resp_rate_score,pao2_aado2_score,hematocrit_score,wbc_score,creatinine_score,uo_score,bun_score,sodium_score,albumin_score,bilirubin_score,glucose_score,acidbase_score,gcs_score,duration,log_duration,over72h,alive96h
0,38910812,EMERGENCY ROOM,Other,ENGLISH,UNKNOWN,SINGLE,M,56,17,77.0,NaN,19.0,18.000000,17.0,7.33,7.305000,7.28,7.4,6.950000,6.5,108.0,82.0,72.440000,65.0,100.0,73.720000,57.0,63.0,63.000000,63.0,26.0,22.800000,20.0,37.06,36.551667,36.00,98.0,94.840000,92.0,136.0,102.400000,62.0,0,0,0,0,1,0,1,50.0,50.000000,50.0,6.0,5.600000,5.0,16.0,16.000000,16.0,0,1.0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,14,3.0,2.0,3.0,1.0,4,1.0,118,5.0,15.0,0.0,6.0,2.0,3.0,0.0,4.0,8.0,11.0,2.0,0.0,8.0,0.0,6.0,48.0,75.033333,4.317932,1,0
1,38388229,EMERGENCY ROOM,Other,ENGLISH,BLACK/AFRICAN AMERICAN,MARRIED,M,81,45,95.5,180.0,23.0,22.500000,22.0,7.44,7.435000,7.43,NaN,NaN,NaN,210.0,110.0,89.333333,54.0,103.0,83.269231,71.0,91.0,80.555556,71.0,33.0,23.940000,16.0,38.61,37.426667,36.67,100.0,98.666667,96.0,205.0,162.666667,109.0,0,0,0,0,1,0,1,50.0,42.500000,40.0,5.0,5.000000,5.0,16.0,15.333333,15.0,0,0.0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,5,NaN,1.0,0.0,0.0,3,1.0,60,5.0,7.0,0.0,6.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,33.0,213.633333,5.364261,1,1
2,31753166,TRANSFER FROM HOSPITAL,Medicare,ENGLISH,WHITE,MARRIED,M,91,73,79.5,175.0,27.0,26.333333,26.0,7.49,7.460000,7.43,NaN,NaN,NaN,300.0,74.0,61.656250,60.0,128.0,72.362500,45.0,95.0,73.681818,45.0,38.0,19.234375,14.0,38.00,37.130000,36.22,100.0,98.156250,94.0,72.0,71.500000,71.0,0,0,0,0,0,0,0,100.0,48.750000,30.0,10.0,5.700000,5.0,24.0,20.916667,20.0,0,0.0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,7,2.0,0.0,NaN,0.0,3,2.0,72,0.0,10.0,0.0,6.0,5.0,3.0,0.0,7.0,4.0,11.0,0.0,NaN,NaN,0.0,2.0,24.0,90.416667,4.504429,1,1
3,30003299,EMERGENCY ROOM,Other,ENGLISH,WHITE,SINGLE,M,26,1,120.0,178.0,29.0,24.888889,21.0,7.40,7.335556,7.27,4.0,2.777778,1.5,280.0,133.0,119.500000,101.0,122.0,93.071429,70.0,NaN,NaN,NaN,18.0,17.105263,12.0,37.44,36.971667,36.39,100.0,98.555556,96.0,185.0,152.166667,130.0,0,0,0,0,0,0,0,50.0,48.333333,40.0,5.0,5.000000,5.0,25.0,23.600000,22.0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0.0,0.0,NaN,0.0,3,0.0,48,7.0,7.0,0.0,0.0,0.0,3.0,0.0,0.0,4.0,0.0,0.0,NaN,NaN,0.0,12.0,15.0,154.183333,5.038142,1,1
4,31166711,EMERGENCY ROOM,Other,ENGLISH,WHITE,SINGLE,M,42,77,97.6,183.0,32.0,20.750000,15.0,7.22,7.156500,7.00,6.4,4.485000,2.2,72.0,150.0,128.500000,113.0,88.0,67.607143,47.0,NaN,NaN,NaN,35.0,16.017857,10.0,39.80,38.150000,37.30,100.0,90.620690,78.0,173.0,120.421053,77.0,1,1,0,1,1,0,4,100.0,100.000000,100.0,16.0,12.769231,10.0,32.0,26.500000,21.0,1,0.0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,14,4.0,1.0,2.0,4.0,1,2.0,92,13.0,15.0,0.0,9.0,0.0,3.0,0.0,7.0,5.0,7.0,2.0,11.0,5.0,0.0,12.0,3.0,420.283333,6.040929,1,1


**Summary statistics**

In [3]:
df_train.describe()

,stay_id,age,hours_in_hosp_before_intubation,weight,height,co2_total_max,co2_total_avg,co2_total_min,ph_max,ph_avg,ph_min,lactate_max,lactate_avg,lactate_min,pao2fio2ratio,heart_rate_max,heart_rate_avg,heart_rate_min,mbp_max,mbp_avg,mbp_min,mbp_ni_max,mbp_ni_avg,mbp_ni_min,resp_rate_max,resp_rate_avg,resp_rate_min,temp_max,temp_avg,temp_min,spo2_max,spo2_avg,spo2_min,glucose_max,glucose_avg,glucose_min,vasopressin,epinephrine,dobutamine,norepinephrine,phenylephrine,dopamine,count_of_vaso,fio2_max,fio2_avg,fio2_min,peep_max,peep_avg,peep_min,plateau_pressure_max,plateau_pressure_avg,plateau_pressure_min,rrt,sinus_rhythm,neuroblocker,congestive_heart_failure,cerebrovascular_disease,dementia,chronic_pulmonary_disease,rheumatic_disease,mild_liver_disease,diabetes_without_cc,diabetes_with_cc,paraplegia,renal_disease,malignant_cancer,severe_liver_disease,metastatic_solid_tumor,aids,SOFA,respiration,coagulation,liver,cardiovascular,cns,renal,apsiii,hr_score,mbp_score,temp_score,resp_rate_score,pao2_aado2_score,hematocrit_score,wbc_score,creatinine_score,uo_score,bun_score,sodium_score,albumin_score,bilirubin_score,glucose_score,acidbase_score,gcs_score,duration,log_duration,over72h,alive96h
count,1.012100e+04,10121.000000,10121.000000,10052.00000,7645.000000,9307.000000,9307.000000,9307.000000,9307.000000,9307.000000,9307.000000,7961.000000,7961.000000,7961.000000,8919.000000,10102.000000,10102.000000,10102.000000,10102.000000,10102.000000,10102.000000,8365.000000,8365.000000,8365.000000,10102.000000,10102.000000,10102.000000,9448.000000,9448.000000,9448.000000,10098.000000,10098.000000,10098.000000,10079.000000,10079.000000,10079.000000,10121.000000,10121.000000,10121.000000,10121.000000,10121.000000,10121.000000,10121.000000,10113.000000,10113.000000,10113.000000,10101.000000,10101.000000,10101.000000,8816.000000,8816.000000,8816.000000,10121.000000,10107.000000,10121.000000,10121.000000,10121.000000,10121.000000,10121.000000,10121.000000,10121.000000,10121.000000,10121.000000,10121.000000,10121.000000,10121.000000,10121.000000,10121.000000,10121.000000,10121.000000,8605.000000,10084.000000,6855.000000,10103.000000,10121.000000,10119.000000,10121.000000,10103.000000,10103.000000,9568.000000,10100.000000,6717.000000,10086.000000,10085.000000,10086.000000,9870.000000,10087.000000,10082.000000,5355.000000,6855.000000,10118.000000,9135.000000,9593.000000,10121.000000,10121.000000,10121.000000,10121.000000
mean,3.500930e+07,63.818793,79.396502,83.35948,169.166014,26.624154,24.747415,22.842914,7.414035,7.365912,7.309904,3.178588,2.466506,1.833380,249.041288,107.520689,86.762397,71.336191,105.551458,76.331033,55.500709,93.967962,74.149098,59.173939,27.385320,19.586702,12.688725,37.742496,37.050551,36.326662,99.768865,97.776535,92.649039,1432.504862,394.997971,110.192975,0.136844,0.076178,0.021737,0.402628,0.274084,0.041794,0.953265,68.694749,50.717608,43.053001,8.234768,6.943523,5.435967,21.248673,19.176749,17.207827,0.088529,0.641338,0.067681,0.287027,0.179528,0.031321,0.262029,0.031222,0.159767,0.216876,0.089220,0.069460,0.204328,0.111254,0.084379,0.047130,0.008102,8.286039,2.294364,0.692582,0.693508,1.993467,2.078846,1.107026,66.353819,4.033752,12.173810,1.238503,4.713168,1.482805,2.880726,1.052851,2.713167,4.873759,5.800040,0.610196,1.922876,1.951860,1.560486,5.795621,17.325237,128.664010,4.454400,0.519711,0.908803
std,2.877692e+06,16.414636,255.690036,26.20366,10.641055,5.720919,5.677617,6.213692,0.072152,0.077016,0.109400,2.912881,2.060752,1.431109,130.009128,21.428800,16.720484,15.994705,28.940976,9.786836,13.839022,20.364397,11.347417,13.068822,6.286378,3.974822,4.110161,0.884026,0.713191,0.969678,0.829207,2.270479,7.500699,34835.522615,8623.867048,38.755421,0.343700,0.265296,0.145831,0.490451,0.446073,0.200129,1.065860,25.014432,12.417794,9.355780,4.002746,2.956503,2.851863,7.677100,5.017432,4.694608,0.284076,0.479632,0.251210,0.452397,0.383813,0.174192,0.439760,0.173926,0.366408,0.412138,0.285076,0.254247,0.403229,0.314462,0.277

**Drop constant variables**

In [4]:
df_train = df_train.loc[:, df_train.apply(pd.Series.nunique) != 1]
df_train.shape

(10121, 103)

### Assign cluster numbers based on severity scores

In [5]:
df_train = utils.cluster_by_severity(df_train)

Using 24 severity scores...
0    3614
1    3266
3    1678
2    1563
Name: cluster, dtype: int64


### Feature selection

In [6]:
features=None
# features = df_train.select_dtypes(np.number).columns[1:-2].tolist()
# features = ["apsiii",
#             "peep_min",
#             "resp_rate_min",
#             "paraplegia",
#             "neuroblocker",
#             "vasopressin",
#             "chronic_pulmonary_disease",
#             "cerebrovascular_disease",
#             "congestive_heart_failure",
#             "diabetes_with_cc",
#             "ph_max"]
# features = ["apsiii",
#             "peep_min",
#             "resp_rate_min",
#             "paraplegia",
#             "neuroblocker",
#             "vasopressin",
#             "height",
#             "chronic_pulmonary_disease",
#             "cerebrovascular_disease",
#             "congestive_heart_failure",
#             "diabetes_with_cc"]
# features = ["heart_rate_max", "heart_rate_min", 
#             "peep_max", "ph_max", 
#             "resp_rate_max", "resp_rate_min", 
#             "spo2_min", "temp_max", "temp_min"]
# features = ["resp_rate_max",
#             "resp_rate_min",
#             "temp_max",
#             "temp_min",
#             "spo2_min",
#             "glucose_max",
#             "mbp_arterial_max",
#             "apsiii",
#             "glucose_min",
#             "heart_rate_min",
#             "heart_rate_max",
#             "ph_max",
#             "co2_total_min",
#             "co2_total_max",
#             "mbp_ni_min",
#             "peep_min"]
# features = ['ph_max', 'spo2_min',
#             'heart_rate_min', 'heart_rate_max', 
#             'resp_rate_min', 'resp_rate_max',
#             'temp_min', 'temp_max', 
#             'glucose_max', 'glucose_min', 
#             'co2_total_max', 'co2_total_min', 
#             'mbp_max', 'mbp_ni_min', 
#             'apsiii', 
#             'peep_max', 'peep_min']

X_train, y_train = utils.get_X_and_y(df_train, features=features, label=label)
print(X_train.shape, y_train.shape)

preprocessor = utils.define_preprocessor(X_train.columns)

(10121, 98) (10121,)


### Model development

In [7]:
# class_names = ("MV <= 72 hours", "MV > 72 hours")
# class_names = ("Bad outcome", "Good outcome")

clfs = (
    LogisticRegression(max_iter=1000),
#     KNeighborsClassifier(),
#     SVC(),
#     DecisionTreeClassifier(),
#     RandomForestClassifier(),
    GradientBoostingClassifier(),
#     CalibratedClassifierCV(GradientBoostingClassifier(), method='isotonic', cv=3)
)

for clf in clfs:
    pipe = Pipeline(steps=[('preprocessor', preprocessor),
                          ('classifier', clf)])
    scores = utils.benchmark_cv_score(pipe, X_train, y_train)

________________________________________________________________________________

Model training: 
LogisticRegression(max_iter=1000)
train time: 3.389s

Average test_Precision: 0.676 (+/- 0.02)
Average test_Recall: 0.676 (+/- 0.02)
Average test_F1: 0.676 (+/- 0.02)
Average test_ROC AUC: 0.740 (+/- 0.03)
Average test_PR AUC: 0.748 (+/- 0.04)
________________________________________________________________________________

Model training: 
GradientBoostingClassifier()
train time: 20.169s

Average test_Precision: 0.692 (+/- 0.02)
Average test_Recall: 0.691 (+/- 0.02)
Average test_F1: 0.691 (+/- 0.02)
Average test_ROC AUC: 0.763 (+/- 0.02)
Average test_PR AUC: 0.772 (+/- 0.02)


In [ ]:
from scipy.stats import mannwhitneyu, ttest_ind
print(mannwhitneyu(scores_S0['test_roc'], scores_S2['test_roc'], alternative="two-sided"))
print(ttest_ind(scores_S0['test_roc'], scores_S2['test_roc']))

### Compare full and reduced models

In [ ]:
X_train, y_train = utils.get_X_and_y(df_train, features=None, label=label)
print(X_train.shape, y_train.shape)

preprocessor = utils.define_preprocessor(X_train.columns)
clf = GradientBoostingClassifier()

pipe = Pipeline(steps=[('preprocessor', preprocessor),
                       ('classifier', clf)])

y_proba_full = utils.benchmark_cv(pipe, X_train, y_train)

In [ ]:
X_train, y_train = utils.get_X_and_y(df_train, features=features, label=label)
print(X_train.shape, y_train.shape)

preprocessor = utils.define_preprocessor(X_train.columns)
clf = GradientBoostingClassifier()

pipe = Pipeline(steps=[('preprocessor', preprocessor),
                       ('classifier', clf)])

y_proba_small = utils.benchmark_cv(pipe, X_train, y_train)

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve

plt.figure();
sns.lineplot(x=[0, 1], y=[0, 1], color=sns.color_palette()[0], lw=2, linestyle='--', label="Chance")

fpr, tpr, _ = roc_curve(y_train, y_proba_full[:,-1])
roc_auc = roc_auc_score(y_train, y_proba_full[:,-1])
sns.lineplot(x=fpr, y=tpr, lw=3, color=sns.color_palette()[1], 
             label="All features: AUC = %0.2f" % roc_auc)

fpr, tpr, _ = roc_curve(y_train, y_proba_small[:,-1])
roc_auc = roc_auc_score(y_train, y_proba_small[:,-1])
sns.lineplot(x=fpr, y=tpr, lw=3, color=sns.color_palette()[2], 
             label="15 features: AUC = %0.2f" % roc_auc)

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC curve")
plt.legend(loc="lower right", fontsize=14);

plt.savefig("../results/Feature selection ROC CV", bbox_inches='tight', dpi=300, transparent=False, pad_inches=0);

### Model calibration

In [ ]:
preprocessor = utils.define_preprocessor(X_train.columns)
clf = GradientBoostingClassifier()
calibrated_clf = CalibratedClassifierCV(clf, method='isotonic', cv=3)

pipe = Pipeline(steps=[('preprocessor', preprocessor),
                       ('classifier', clf)])
calibrated_pipe = Pipeline(steps=[('preprocessor', preprocessor),
                       ('classifier', calibrated_clf)])

**Run cross validation to calibrate the model**

In [ ]:
y_proba = utils.benchmark_cv(pipe, X_train, y_train)

In [ ]:
y_proba_c = utils.benchmark_cv(calibrated_pipe, X_train, y_train)

**Diagnostic plots**

In [ ]:
sns.lineplot(x=[0, 1], y=[0, 1], 
             color=sns.color_palette()[0], 
             lw=2, linestyle='--', 
             label="Perfectly calibrated")

fop, mpv = calibration_curve(y_train, y_proba[:,1], n_bins=30, normalize=False)
sns.lineplot(x=mpv, y=fop, 
             lw=3, marker='.', markersize=15, 
             color=sns.color_palette()[1],
             label="Uncalibrated");

fop, mpv = calibration_curve(y_train, y_proba_c[:,1], n_bins=30, normalize=False)
sns.lineplot(x=mpv, y=fop, 
             lw=3, marker='.', markersize=15, 
             color=sns.color_palette()[2],
             label="Calibrated");

plt.legend(fontsize=16, loc="upper left");
plt.xlabel("Mean predicted value");
plt.ylabel("Fraction of positives");

plt.savefig("../results/15ft_calibration.png", bbox_inches='tight', dpi=300, pad_inches=0);

In [ ]:
sns.histplot(y_proba[:,1], bins=10, stat="count", 
             color=sns.color_palette()[1], lw=3, fill=False, 
             label="Uncalibrated");
sns.histplot(y_proba_c[:,1], bins=10, stat="count", 
             color=sns.color_palette()[2], lw=3, fill=False, 
             label="Calibrated");
plt.ylim([0, 3800]);
plt.legend(fontsize=16, loc="upper right");
plt.xlabel("Mean predicted value");

plt.savefig("../results/15ft_probabilities.png", bbox_inches='tight', dpi=300, pad_inches=0);

### Threshold selection

In [8]:
def select_threshold(y_train, y_proba):
    precision, recall, thresholds = precision_recall_curve(y_train, y_proba)
    fscore = (2 * precision * recall) / (precision + recall)
    idx = np.argmax(fscore)
    thresh = thresholds[idx]
    print('Best threshold is %.3f, F1 score=%.3f' % (thresh, fscore[idx]))
    return thresh

In [9]:
preprocessor = utils.define_preprocessor(X_train.columns)
clf = GradientBoostingClassifier()

pipe = Pipeline(steps=[('preprocessor', preprocessor),
                       ('classifier', clf)])
y_proba = utils.benchmark_cv(pipe, X_train, y_train)

________________________________________________________________________________

Model training: 
train time: 68.685s


In [ ]:
thresh = select_threshold(y_train, y_proba)

In [ ]:
df_train["y_proba"] = y_proba[:,1]

In [ ]:
select_threshold(df_train[df_train.cluster==3].over72h, df_train[df_train.cluster==3].y_proba)

### Evaluation using CV

In [ ]:
df_train["y_pred"] = utils.evaluate_model(y_train, y_proba, ("MV < 72h", "MV >= 72h"), 
                                          "CV, cluster 3", thresh=thresh, digits=3)

In [ ]:
df_train["outcome"] = 0
df_train.loc[(df_train.over72h == 0) & (df_train.y_pred == 0), "outcome"] = "TN"
df_train.loc[(df_train.over72h == 1) & (df_train.y_pred == 0), "outcome"] = "FN"
df_train.loc[(df_train.over72h == 0) & (df_train.y_pred == 1), "outcome"] = "FP"
df_train.loc[(df_train.over72h == 1) & (df_train.y_pred == 1), "outcome"] = "TP"
df_train.outcome.value_counts()

In [ ]:
tmp = pd.DataFrame((df_train.groupby("cluster").outcome.value_counts() / 
                    df_train.groupby('cluster').size() * 100).unstack())
tmp

In [ ]:
color = sns.color_palette("Set1")

tmp.plot(kind="bar", stacked=True, color=color, alpha=0.8);
plt.legend(bbox_to_anchor=(1, 0.5), fontsize=16);

In [ ]:
from sklearn.metrics import classification_report
# Cluster 1
print(classification_report(df_train[df_train.cluster==0].over72h, df_train[df_train.cluster==0].y_pred, digits=3))
print(classification_report(df_train[df_train.cluster==1].over72h, df_train[df_train.cluster==1].y_pred, digits=3))
print(classification_report(df_train[df_train.cluster==2].over72h, df_train[df_train.cluster==2].y_pred, digits=3))
print(classification_report(df_train[df_train.cluster==3].over72h, df_train[df_train.cluster==3].y_pred, digits=3))

### Model evaluation on MIMIC data

In [ ]:
preprocessor = utils.define_preprocessor(X_train.columns)
clf = GradientBoostingClassifier()

pipe = Pipeline(steps=[('preprocessor', preprocessor),
                       ('classifier', clf)])
pipe.fit(X_train, y_train)

**Feature importance**

In [ ]:
feature_weights = pd.DataFrame(zip(X_train.columns, pipe['classifier'].feature_importances_), 
                               columns=["feature", "weight"]).sort_values(by="weight", ascending=False)

plt.rcParams['figure.figsize'] = (4, 6)

ax = sns.barplot(y="feature", x="weight", data=feature_weights, orient="h");

plt.ylabel("Feature");
plt.xlabel("Relative importance");
plt.xlim([0, 0.35]);

utils.show_values_on_bars(ax, orient="h", space=0.01)

plt.savefig("../results/Feature importance", bbox_inches='tight', dpi=300, transparent=False, pad_inches=0);

In [ ]:
feature_weights.feature.tolist()

**Test set**

In [ ]:
df_test = pd.read_csv("../data/mimic-emr-test-S0.csv")
df_test.drop(columns=["starttime", "endtime"], inplace=True)

print(df_test.shape)
df_test.head()

In [ ]:
df_test = df_test.loc[:, df_test.apply(pd.Series.nunique) != 1]
df_test.shape

In [ ]:
X_test, y_test = utils.get_X_and_y(df_test, features=features, label=label)
print(X_test.shape, y_test.shape)

In [ ]:
y_proba_test = pipe.predict_proba(X_test)
utils.evaluate_model(y_test, y_proba_test, ("MV < 72h", "MV >= 72h"), "test", digits=3, 
                     save_figures=False, filename="../results/mimic-test")

### External validation on eICU data

In [ ]:
df_eicu = pd.read_csv("../data/eicu-ft17.csv")
print(df_eicu.shape)
df_eicu.head()

In [ ]:
df_eicu.over72h.value_counts()

In [ ]:
df_eicu.rename({"mbp_arterial_max": "mbp_max"}, axis=1, inplace=True)

In [ ]:
X_eicu, y_eicu = utils.get_X_and_y(df_eicu, features=features, label=label)
print(X_eicu.shape, y_eicu.shape)

In [ ]:
y_proba_eicu = pipe.predict_proba(X_eicu)
utils.evaluate_model(y_eicu, y_proba_eicu, ("MV < 72h", "MV >= 72h"), "eICU", digits=3, 
                     save_figures=False, filename="../results/eicu")

In [ ]:
from sklearn.metrics import f1_score, auc, roc_auc_score
roc_auc = roc_auc_score(y_eicu, y_proba_eicu[:,-1])
roc_auc

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve

plt.figure();
sns.lineplot(x=[0, 1], y=[0, 1], color=sns.color_palette()[0], lw=2, linestyle='--', label="Chance")

# fpr, tpr, _ = roc_curve(y_test, y_proba_test[:,-1])
# roc_auc = roc_auc_score(y_test, y_proba_test[:,-1])
# sns.lineplot(x=fpr, y=tpr, lw=3, color=sns.color_palette()[1], 
#              label="MIMIC-IV: AUC = %0.2f" % roc_auc)

fpr, tpr, _ = roc_curve(y_eicu, y_proba_eicu[:,-1])
roc_auc = roc_auc_score(y_eicu, y_proba_eicu[:,-1])
sns.lineplot(x=fpr, y=tpr, lw=3, color=sns.color_palette()[2], 
             label="eICU: AUC = %0.2f" % roc_auc)

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC curve")
plt.legend(loc="lower right", fontsize=14);

# plt.savefig("../results/ROC mimic vs eicu", bbox_inches='tight', dpi=300, pad_inches=0);